In [1]:
from cnn_method import *
from cnn_preproc_function import *
from inference_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os
import shutil

import glob
import joblib

asset_model_path = "//OFC-THR-LWS/DLOutput/_Done_Accepted/Done_Accepted_Y001_10_BS/Y001B_D0H0M15_B/ASSETS_MODELS/DIA_1542226633"

Using TensorFlow backend.


In [3]:
DATA_PARAMS = joblib.load(os.path.join(asset_model_path,"DATA_PARAMS.pkl"))
MODEL_PARAMS = joblib.load(os.path.join(asset_model_path,"MODEL_PARAMS.pkl"))

In [4]:
MODEL_PARAMS

{'ASSET_FOLDER_NAME': 'DIA_1542226633',
 'BATCH_SIZE': 256,
 'EPOCHS': 900,
 'INIT_TIME': '1542226633',
 'LEARNING_RATE': 2.5e-05,
 'collect_bestmodel_folder': '//OFC-THR-LWS/DLOutput\\Y001B_D0H0M15_B\\BESTMODEL',
 'collect_signal_folder': '//OFC-THR-LWS/DLOutput\\Y001B_D0H0M15_B\\SIGNAL',
 'device': '0',
 'keras_model_function': 'rnn_model_conf_1_best',
 'logs_folder': '//OFC-THR-LWS/DLOutput\\Y001B_D0H0M15_B\\ASSETS_MODELS\\DIA_1542226633\\logs',
 'models_folder': '//OFC-THR-LWS/DLOutput\\Y001B_D0H0M15_B\\ASSETS_MODELS\\DIA_1542226633\\models',
 'outputmain_folder': '//OFC-THR-LWS/DLOutput\\Y001B_D0H0M15_B\\ASSETS_MODELS',
 'project_folder': '//OFC-THR-LWS/DLOutput\\Y001B_D0H0M15_B\\ASSETS_MODELS\\DIA_1542226633',
 'root_output_folder': '//OFC-THR-LWS/DLOutput',
 'training_code': 'Y001B_D0H0M15_B',
 'training_number': 'Y001B'}

In [5]:
DATA_PARAMS


{'BREAKOUT_WINDOW': 30,
 'CLEAN_METHOD_X': 'breakout_only_x',
 'FLIP': False,
 'FUTURE_PERIOD_PREDICT': 10,
 'SEQ_LEN': 30,
 'TARGET_FUNCTION': 'cumulative_returns',
 'TARGET_THRESHOLD': 0.001,
 'TARGET_TO_PREDICT': 'DIA',
 'TimeFrame': 'D0H0M15',
 'end_split': [datetime.datetime(2011, 1, 1, 0, 0),
  datetime.datetime(2013, 1, 1, 0, 0),
  datetime.datetime(2017, 1, 1, 0, 0)],
 'possible_assets': array(['EURUSD', 'EEM', 'EFA', 'EWZ', 'FXI', 'GDX', 'HYG', 'IAU', 'IWM',
        'SPY', 'USO', 'VWO', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU',
        'XOP', 'JNK', 'IYR', 'VEA', 'SLV', 'XLV', 'RSX', 'TLT', 'EWJ',
        'OIH', 'GLD', 'EZU', 'KRE', 'SMH', 'XLB', 'XRT', 'LQD', 'EWT',
        'XLY', 'VNQ', 'EWH', 'EWW', 'XBI', 'DIA', 'EWG', 'VGK', 'IEF',
        'EMB', 'FEZ', 'AGG', 'EWY', 'ITB', 'XHB', 'TBT', 'DBC', 'IBB',
        'EWC', 'UNG', 'SHY', 'PFF', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCHF',
        'USDCAD', 'EURJPY', 'EURGBP', 'NZDUSD', 'XAGUSD', 'XAUUSD'],
       dtype=object),
 'raw_

In [6]:
raw_data_file = DATA_PARAMS["raw_data_file"]
TARGET_TO_PREDICT = DATA_PARAMS["TARGET_TO_PREDICT"]
BREAKOUT_WINDOW = DATA_PARAMS["BREAKOUT_WINDOW"]
SEQ_LEN = DATA_PARAMS["SEQ_LEN"]
try:
    CLEAN_METHOD_X = DATA_PARAMS["CLEAN_METHOD_X"]
except:
    print("CLEAN_METHOD_X default to breakout_only_x")
    CLEAN_METHOD_X = "breakout_only_x"


In [7]:
# gpudevice = str(MODEL_PARAMS["device"])
gpudevice = "0"
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]=gpudevice
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [29]:
raw_data_file

'DATA\\PRICE_LIQUIDASSET_15_MIN.csv'

In [30]:
df = load_data(raw_data_file)
df2 = df.copy()

Load Data: Done!


In [33]:
df2.columns
df2.head

<bound method NDFrame.head of                       EURUSD    EEM    EFA    EWZ    FXI    GDX    HYG    IAU  \
Date                                                                            
2008-05-04 18:15:00  1.54340    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 18:30:00  1.54350    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 18:45:00  1.54270    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 19:00:00  1.54320    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 19:15:00  1.54360    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 19:30:00  1.54350    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 19:45:00  1.54340    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 20:00:00  1.54370    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 20:15:00  1.54460    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2008-05-04 20:30:00  1.54470    NaN    NaN    NaN    NaN    NaN    NaN    NaN  

In [11]:
df = clean_data_x(df, target_col = TARGET_TO_PREDICT, window = BREAKOUT_WINDOW, method = CLEAN_METHOD_X)

In [12]:
df.dropna(inplace = True)
df

,EURUSD,EEM,EFA,EWZ,FXI,GDX,HYG,IAU,IWM,SPY,...,GBPUSD,AUDUSD,USDCHF,USDCAD,EURJPY,EURGBP,NZDUSD,XAGUSD,XAUUSD,target
Date,,,,,,,,,,,,,,,,,,,,,
2008-07-01 10:30:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.002129
2008-07-01 10:45:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.001682
2008-07-01 11:00:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.001944
2008-07-01 11:15:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000706
2008-07-01 11:30:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,-1.0,0.0,0.0,-0.002996
2008-07-01 11:45:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.001326
2008-07-01 12:00:00,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002390
2008-07-01 12:15:00,1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.003993
2008-07-01 12:30:00,1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.0,-1.0,-1.0,...,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,1.0,1.0,-0.003385


In [13]:
df2 = df.copy()

In [14]:
df.index

DatetimeIndex(['2008-07-01 10:30:00', '2008-07-01 10:45:00',
               '2008-07-01 11:00:00', '2008-07-01 11:15:00',
               '2008-07-01 11:30:00', '2008-07-01 11:45:00',
               '2008-07-01 12:00:00', '2008-07-01 12:15:00',
               '2008-07-01 12:30:00', '2008-07-01 12:45:00',
               ...
               '2018-10-31 13:45:00', '2018-10-31 14:00:00',
               '2018-10-31 14:15:00', '2018-10-31 14:30:00',
               '2018-10-31 14:45:00', '2018-10-31 15:00:00',
               '2018-10-31 15:15:00', '2018-10-31 15:30:00',
               '2018-10-31 15:45:00', '2018-10-31 16:00:00'],
              dtype='datetime64[ns]', name='Date', length=69834, freq=None)

In [15]:
target_col= "target"
x_columns = [j for j in df.columns if j != target_col]

#Scaling
scaler = joblib.load(os.path.join(asset_model_path, "scaler.pkl"))
df.loc[:,x_columns] = scaler.transform(df[x_columns])

X = df[x_columns].values
Y = df[target_col].values


In [16]:
X.shape, Y.shape

((69834, 68), (69834,))

In [18]:
model_files = glob.glob(os.path.join(asset_model_path,"*.model"))
model_files

['//OFC-THR-LWS/DLOutput/_Done_Accepted/Done_Accepted_Y001_10_BS/Y001B_D0H0M15_B/ASSETS_MODELS/DIA_1542226633\\DL-0033-0.7301-0.5566-0.4469-0.4196.model']

In [19]:
best_model_path = os.path.join(asset_model_path,model_files[0])
model = keras.models.load_model(best_model_path, custom_objects=None, compile=False)

In [20]:
data_gen = TimeseriesGenerator(X,Y,
                       length=SEQ_LEN, sampling_rate=1,
                       batch_size=128,
                       shuffle=False)
                       

In [21]:
y_pred = model.predict_generator(data_gen)

In [25]:
test_df = df[["target"]].copy()
diff = test_df.shape[0] - len(y_pred)
test_df = test_df[diff:]
test_df["signal"] = y_pred

In [28]:
test_df.to_csv("C:/Users/Workstation/Desktop/test.csv")